In [ ]:
!pip install transformers datasets torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

In [ ]:
import json
import pandas as pd
from datasets import Dataset

# Load the JSON data
with open('bartintent.json') as f:
    data = json.load(f)

# Convert to pandas DataFrame
df = pd.DataFrame(data)

# Create a Dataset object
dataset = Dataset.from_pandas(df)


In [ ]:
# Split the dataset into train and eval sets
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

def preprocess_function(examples):
    inputs = examples["Question"]
    targets = examples["Query"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Map:   0%|          | 0/1171 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/293 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
# Load the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=3e-5,  # Adjusted learning rate
    per_device_train_batch_size=4,  # Increased batch size
    per_device_eval_batch_size=4,  # Increased batch size
    num_train_epochs=5,  # Increased epochs
    weight_decay=0.01,
    logging_steps=500,  # Log every 500 steps
)
# Create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator,
)
# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.236706
2,0.696100,0.154253
3,0.696100,0.116141
4,0.199300,0.100979
5,0.199300,0.092074


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=1465, training_loss=0.34620010063509893, metrics={'train_runtime': 9204.3515, 'train_samples_per_second': 0.636, 'train_steps_per_second': 0.159, 'total_flos': 59867903047680.0, 'train_loss': 0.34620010063509893, 'epoch': 5.0})

In [ ]:
# Save the model
model.save_pretrained("./fine-tuned-bart")
tokenizer.save_pretrained("./fine-tuned-bart")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine-tuned-bart/tokenizer_config.json',
 './fine-tuned-bart/special_tokens_map.json',
 './fine-tuned-bart/vocab.json',
 './fine-tuned-bart/merges.txt',
 './fine-tuned-bart/added_tokens.json')

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

# Function to generate MongoDB query
def generate_query(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=512, truncation=True, padding="max_length")
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return query

# Example usage
question = "What is the most recent medication prescribed to Emily Green?"
query = generate_query(question)
print(query)


db.patients.find({"name.first": "Emily", "name.last": "Green"}, {"medical_history.medications": 1, "_id": 0})


In [ ]:
!pip install nltk rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=53792828dd8f2147c34eedf3f951704401396c36b99f281685c0e065fc5887f5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

def generate_query_finetuned(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=512, truncation=True, padding="max_length")
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return query



In [ ]:
test_data = [

    {'input': 'What surgeries has Charlie Smith had in the past?', 'expected_query': '{"db.patients.find({ "name.first": "Charlie", "name.last": "Smith" }, { "medical_history.surgeries": 1, "_id": 0"}'},

    {'input': 'What is the insurance provider for David Smith?', 'expected_query': '{"db.patients.find({ "name.first": "David", "name.last": "Smith" }, { "insurance.provider": 1, "_id": 0"}'},

    {'input': 'What is the marital status of Charlie Smith?', 'expected_query': '{"db.patients.find({ "name.first": "Charlie", "name.last": "Smith" }, { "marital_status": 1, "_id": 0"}'},

    {'input': 'What is the contact phone number for Eve Smith?', 'expected_query': '{"db.patients.find({ "name.first": "Eve", "name.last": "Smith" }, { "contact.phone": 1, "_id": 0"}'},

    {'input': "How many patients have a blood group of 'B+'?", 'expected_query': 'db.patients.aggregate([{ "$match": { "blood_group": "B+" } }, { "$group": { "_id": null, "totalCount": { "$sum": 1 } } }, { "$project": { "_id": 0, "totalCount": 1 } }])'},

    {'input': "How many patients have a blood group of 'AB+'?", 'expected_query': 'db.patients.aggregate([{ "$match": { "blood_group": "AB+" } }, { "$group": { "_id": null, "totalCount": { "$sum": 1 } } }, { "$project": { "_id": 0, "totalCount": 1 } }])'},

    {'input': "How many patients have had knee replacement surgery?", 'expected_query': 'db.patients.countDocuments({ "medical_history.surgeries.name": "Knee Replacement" })'},

    {'input': "Find the contact details for patient with ID '11223'?", 'expected_query': 'db.patients.find({ "id": "11223" }, { "contact": 1, "_id": 0 })'},

    {'input': "What is the doctor’s name for the last appointment of the patient with ID 31011?", 'expected_query': 'db.patients.aggregate([{ "$match": { "id": "31011" } }, { "$unwind": "$appointments" }, { "$sort": { "appointments.date": -1 } }, { "$limit": 1 }, { "$project": { "_id": 0, "appointments.doctor.name": 1 } }])'},

    {'input': "What is the doctor’s name for the last appointment of the patient with ID 31012?", 'expected_query': 'db.patients.aggregate([{ "$match": { "id": "31012" } }, { "$unwind": "$appointments" }, { "$sort": { "appointments.date": -1 } }, { "$limit": 1 }, { "$project": { "_id": 0, "appointments.doctor.name": 1 } }])'},

    {'input': "What is the minimum age in the records?", 'expected_query': 'db.patients.aggregate([{ "$sort": { "age": 1 } }, { "$limit": 1 }, { "$project": { "name": 1, "_id": 0 } }])'},

    {'input': "What is the maximum age in the records?", 'expected_query': 'db.patients.aggregate([{ "$sort": { "age": -1 } }, { "$limit": 1 }, { "$project": { "name": 1, "_id": 0 } }])'},

    {'input': "Provide the contact address for Harper Hall.", 'expected_query': 'db.patients.find({ "name.first": "Harper", "name.last": "Hall" }, { "contact.address": 1, "_id": 0 })'},

    {'input': "What is the status of the insurance claims for Ethan Allen?", 'expected_query': 'db.patients.find({ "name.first": "Ethan", "name.last": "Allen" }, { "insurance.claimed_insurance.status": 1, "_id": 0 })'},

    {'input': "What surgeries has David Jones had in the past?", 'expected_query': 'db.patients.find({ "name.first": "David", "name.last": "Jones" }, { "medical_history.surgeries": 1, "_id": 0 })'},

    {'input': "What medications is Olivia Taylor currently taking?", 'expected_query': 'db.patients.find({ "name.first": "Olivia", "name.last": "Taylor" }, { "medications": 1, "_id": 0 })'},

    {'input': "How many patients are diagnosed with 'Diabetes'?", 'expected_query': 'db.patients.aggregate([{ "$match": { "medical_history.conditions.name": "Diabetes" } }, { "$group": { "_id": null, "totalCount": { "$sum": 1 } } }, { "$project": { "_id": 0, "totalCount": 1 } }])'},

    {'input': 'What is the marital status of Charlie Smith?', 'expected_query': '{"db.patients.find({ "name.first": "Charlie", "name.last": "Smith" }, { "marital_status": 1, "_id": 0"}'},

    {'input': 'What is the contact phone number for Eve Smith?', 'expected_query': '{"db.patients.find({ "name.first": "Eve", "name.last": "Smith" }, { "contact.phone": 1, "_id": 0"}'},

    {'input': 'What surgeries has Olivia Smith had in the past?', 'expected_query': '{"db.patients.find({ "name.first": "Olivia", "name.last": "Smith" }, { "medical_history.surgeries": 1, "_id": 0"}'},

    {'input': 'How many patients have a blood group A+?','expected_query':'{"db.patients.find({ "blood_group": "A+" }, { "name": 1, "id": 1, "age": 1, "blood_group": 1, "_id": 0 }'},

    {'input': 'Get the health monitoring data for the patient named Zoe Harris.', 'expected_query': 'db.patients.find({ "name.first": "Zoe", "name.last": "Harris" }, { "health_monitoring": 1, "_id": 0 })'},

    {'input': 'List the medical history for the patient named James Brown.', 'expected_query': 'db.patients.find({ "name.first": "James", "name.last": "Brown" }, { "medical_history": 1, "_id": 0 })'},

    {'input': 'Retrieve all insurance details for the patient named Mia Thompson.', 'expected_query': 'db.patients.find({ "name.first": "Mia", "name.last": "Thompson" }, { "insurance": 1, "_id": 0 })'},

    {'input': 'Get the contact information for the patient named Sebastian Davis.', 'expected_query': 'db.patients.find({ "name.first": "Sebastian", "name.last": "Davis" }, { "contact": 1, "_id": 0 })'},

    {'input': 'List all the appointments for the patient named James Smith.', 'expected_query': 'db.patients.aggregate([ { "$match": { "name.first": "James", "name.last": "Smith" } }, { "$unwind": "$appointments" }, { "$project": { "_id": 0, "appointments.date": 1, "appointments.type": 1, "appointments.doctor": 1 } } ])'},

]


In [ ]:
# Initialize metrics
bleu_scores = []
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
precisions = []
partial_matches = 0  # For Partial Match Accuracy

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Download necessary NLTK data
nltk.download('punkt')

# Define function for Partial Match Accuracy
def partial_match_accuracy(expected_query, generated_query, threshold=0.5):
    expected_tokens = set(nltk.word_tokenize(expected_query))
    generated_tokens = set(nltk.word_tokenize(generated_query))
    common_tokens = len(expected_tokens.intersection(generated_tokens))
    return common_tokens / len(expected_tokens) >= threshold

# Evaluation
for data in test_data:
    generated_query = generate_query_finetuned(data['input'])

    # Tokenize and prepare sets
    reference_tokens = nltk.word_tokenize(data['expected_query'])
    hypothesis_tokens = nltk.word_tokenize(generated_query)
    reference_set = set(reference_tokens)
    hypothesis_set = set(hypothesis_tokens)

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], hypothesis_tokens)
    bleu_scores.append(bleu_score)

    # Calculate ROUGE scores
    rouge_score = scorer.score(data['expected_query'], generated_query)
    rouge_scores['rouge1'].append(rouge_score['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(rouge_score['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(rouge_score['rougeL'].fmeasure)

    # Calculate Precision
    true_positives = len(reference_set.intersection(hypothesis_set))
    precision = true_positives / len(hypothesis_set) if hypothesis_set else 0
    precisions.append(precision)

# Average metrics
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])
avg_precision = sum(precisions) / len(precisions)

print(f'Average BLEU Score: {avg_bleu}')
print(f'Average ROUGE-1 Score: {avg_rouge1}')
print(f'Average ROUGE-2 Score: {avg_rouge2}')
print(f'Average ROUGE-L Score: {avg_rougeL}')
print(f'Average Precision: {avg_precision}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Average BLEU Score: 0.731689642157723
Average ROUGE-1 Score: 0.8421773711044398
Average ROUGE-2 Score: 0.7890287431401488
Average ROUGE-L Score: 0.8308015064677057
Average Precision: 0.9034973604826546


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/fine-tuned-bart/content/drive/MyDrive/

cp: missing destination file operand after '/content/fine-tuned-bart/content/drive/MyDrive/'
Try 'cp --help' for more information.
